In [1]:
# ! pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai faiss-cpu unstructured

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
loader = DirectoryLoader("../city_data")
docs = loader.load_and_split(text_splitter=text_splitter)

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [4]:
from langchain_community.vectorstores import ApertureDB
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vector_db = ApertureDB.from_documents(docs, embeddings)

In [5]:
retriever = vector_db.as_retriever()

In [6]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_cities",
    "Searches and returns excerpts from Wikipedia entries of many cities.",
)
tools = [tool]

In [7]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

/Users/yujian/Documents/workspace/ai_agents_cookbooks/py312/lib/python3.12/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [9]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [12]:
result = agent_executor.invoke(
    {
        "input": "What is the population of Seattle?"
    }
)

In [13]:
result["output"]

"The population of Seattle is approximately 749,256 as of 2022. It is the most populous city in both the state of Washington and the Pacific Northwest region of North America. The Seattle metropolitan area's population is 4.02 million, making it the 15th-largest in the United States."